In [1]:
%pip install --upgrade pip
%pip install --upgrade transformers datasets[audio] accelerate
%pip install -U transformers accelerate datasets[audio] torchaudio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 45.7 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
#ffmpeg 활용 git의 Codespace 내에서 실행 가능
!sudo apt-get update
!sudo apt-get install -y ffmpeg
!ffmpeg -version
%pip install -U ipywidgets jupyterlab_widgets

Hit:1 https://packages.microsoft.com/repos/microsoft-ubuntu-noble-prod noble InRelease
Hit:2 https://dl.yarnpkg.com/debian stable InRelease
Hit:3 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu noble InRelease                         
Hit:5 http://security.ubuntu.com/ubuntu noble-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu noble-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:6.1.1-3ubuntu5).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-g

In [ ]:
from fastapi import FastAPI
from fastapi.responses import JSONResponse
app = FastAPI()
@app.get("/")
def root(): return JSONResponse({"ok": True})

In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# from datasets import load_dataset

# PyTorch에서 사용할 디바이스와 데이터 타입 설정
device = "cuda:0" if torch.cuda.is_available() else "cpu"   # GPU가 있으면 첫 번째 CUDA GPU, 없으면 CPU
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32  # GPU면 반정밀(속도·메모리 절약), CPU면 일반 정밀, float16이면 연산 속도가 올라감

# 허깅페이스 모델 식별자(Whisper Large v3 Turbo) — 음성 인식용 최신/고속 버전
model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    dtype=dtype,
    device=device,
    return_timestamps=True,   # 청크별로 타임스탬프 반환
    chunk_length_s=10,  # 입력 오디오 10초씩 나누기r
    stride_length_s=2,  # 2초씩 겹치도록 청크 나누기
) 

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]
sample = "/workspaces/LLM_PRACITCE/chap5/lsy_audio_2023_58s.mp3"

result = pipe(sample)
# print(result["text"])

print(result)

`torch_dtype` is deprecated! Use `dtype` instead!


: 

In [1]:
# ✅ CPU 안전 실행 스니펫 (VS Code Codespaces용)
import os
os.environ["OMP_NUM_THREADS"] = "1"   # CPU 스레드 과점유 방지
os.environ["MKL_NUM_THREADS"] = "1"

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cpu"               # Codespaces는 보통 GPU 없음
dtype  = torch.float32       # CPU는 float32가 안전

# 🔽 작은 모델부터: small -> base -> medium 순으로 점진 상향
model_id = "openai/whisper-small"      # or "openai/whisper-base"

# 모델 로드 (from_pretrained의 torch_dtype는 여전히 유효)
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True,
)
processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    device=device,            # "cpu"
    dtype=dtype,              # ← pipeline에는 dtype 사용 (중요)
    return_timestamps=True,
    chunk_length_s=8,         # CPU면 짧게(5~8 권장)
    stride_length_s=2,
    generate_kwargs={
        # 한국어 힌트(언어 감지 시간 단축). 다른 언어면 변경
        "language": "ko",     # "korean" 또는 "ko-KR"도 동작
        "task": "transcribe", # 번역 말고 원문 전사
    },
)

sample = "/workspaces/LLM_PRACITCE/chap5/lsy_audio_2023_58s.mp3"
print(pipe(sample))


`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


: 

In [ ]:
from fastapi import FastAPI
from fastapi.responses import JSONResponse
app = FastAPI()
@app.get("/")
def root(): return JSONResponse({"ok": True})